In [ ]:

from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, length, lit, concat, to_timestamp
from snowflake.snowpark.types import (
    StructType, StructField, IntegerType, DoubleType, StringType, 
    TimestampType, DateType, LongType, FloatType
)
from snowflake.snowpark import functions as F
from snowflake.snowpark.window import Window


from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:

session.use_warehouse("COMPUTE_WH")
credits_csv_file = "@MY_GCS_STAGE/credit_card_fraud.csv"


df = session.read.option("FIELD_DELIMITER", ",").csv(credits_csv_file)
df.printSchema()

Data Type Transformations

In [ ]:
# Define schema
schema = StructType([
    StructField("_c0", IntegerType(), True),
    StructField("ssn", StringType(), True),
    StructField("cc_num", StringType(), True),
    StructField("first", StringType(), True),
    StructField("last", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("street", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("zip", StringType(), True),
    StructField("lat", FloatType(), True),
    StructField("long", FloatType(), True),
    StructField("city_pop", LongType(), True),
    StructField("job", StringType(), True),
    StructField("dob", DateType(), True),
    StructField("acct_num", StringType(), True),
    StructField("profile", StringType(), True),
    StructField("trans_num", StringType(), True),
    StructField("trans_date", StringType(), True),  # Read as string initially
    StructField("trans_time", StringType(), True),  # Read as string initially
    StructField("unix_time", LongType(), True),
    StructField("category", StringType(), True),
    StructField("amt", DoubleType(), True),
    StructField("is_fraud", IntegerType(), True),
    StructField("merchant", StringType(), True),
    StructField("merch_lat", DoubleType(), True),
    StructField("merch_long", DoubleType(), True)
])


df = session.read.options({
    "FIELD_OPTIONALLY_ENCLOSED_BY": '"',
    "SKIP_HEADER": 1
}).schema(schema).csv(credits_csv_file)

df.printSchema()

# Concatenating `trans_date` and `trans_time` and convert to Timestamp
df = df.with_column("trans_time", to_timestamp(concat(col("trans_date"), lit(" "), col("trans_time")), "YYYY-MM-DD HH24:MI:SS"))

df.show(5)


In [ ]:
df.count()

Duplicate Check

In [ ]:
partition = Window.partition_by(*df.columns)

duplicates = df.select("*", F.count("*").over(partition).alias("duplicate_records")) \
                .filter(F.col("duplicate_records") > 1)

duplicates.show()

Completeness Check

In [ ]:
total_rows = df.count()
completeness_check = df.select(
    *[
        (F.count(F.when(F.col(c).isNull(), 1)) / total_rows).alias(f"{c}_null_ratio")
        for c in df.columns
    ]
)

completeness_check.show()

Length Checks

In [ ]:
def length_check(df, col_name, required_length=None, min_length=None, max_length=None):
    if required_length:
        return df.filter(length(col(col_name)) == required_length)
    elif min_length is not None and max_length is not None:
        return df.filter(length(col(col_name)).between(min_length, max_length))
    else:
        return df

df_new = length_check(df, 'cc_num', required_length=16)
df_new = length_check(df_new, 'ssn', required_length=11)
df_new = length_check(df_new, 'acct_num', min_length=8, max_length=12)

df_new.show(5)


In [ ]:
df_new.count()

In [ ]:
df_new.write.mode("append").save_as_table("CREDIT_CARD_FRAUD_INTERNAL")


In [ ]:
df.schema["your_column"].datatype